In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import chain
from collections import defaultdict

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def conjuntos_sobre_umbral_sets(tuplas, conjuntos, frec_min):
    '''
    tuplas: lista de conjuntos de varaibles (tuplas de la base de datos)
    conjuntos: lista con conjuntos de variables
    frec_min: int, umbral minimo para seguir en la proxima iteración 
    '''
    nuevas_tuplas = []
    sobrevivientes = []
    muertes = []
    
    diccionario = defaultdict(int)
    posiciones = defaultdict(list)
    
    #t = datetime.now()
    '''for conjunto in conjuntos:
        for tupla in tuplas:
            se_queda = False
            if conjunto.issubset(tupla):
                diccionario[tuple(conjunto)] += 1
                posiciones[tupla].append(conjunto)
                se_queda = True
                #if tupla not in nuevas_tuplas:
                 #   nuevas_tuplas.append(tupla)'''
    for tupla in tuplas:
        se_queda = False
        for conjunto in conjuntos:
            if conjunto.issubset(tupla):
                diccionario[tuple(conjunto)] += 1
                posiciones[tupla].append(conjunto)
                se_queda = True
        if se_queda:
            nuevas_tuplas.append(tupla)
            

    s = filter(lambda x: x[1] >= frec_min, diccionario.items())
    m = filter(lambda x: x[1] < frec_min, diccionario.items())
    
    sobrevivientes = [{*x[0]} for x in s]
    muertes = [{*x[0]} for x in m]
    
    #print('1.- for de sobre umbral tardo {}\n'.format((datetime.now() - t).total_seconds()))            
    #print('murio:', conjunto)
    #print('\nMe demoré {} seg en el sobre umbral'.format((datetime.now() - t).total_seconds()))
    #print()
    return nuevas_tuplas, sobrevivientes, muertes
    

In [3]:
def join_conjunto_sets(conjuntos, muertes):
    nuevo_conjunto = []
    for A in (conjuntos):
        for B in conjuntos[conjuntos.index(A) + 1:]:
            #if set(list(A)[:-1]) == (set(list()[:-1])):
             
            posible = A.union(B)
            try:
                if len(posible) == len(conjuntos[0]) + 1:
                #if len(A.intersection(B)) == len(conjuntos[0]) - 1:
                    #print(posible)
                    if posible not in nuevo_conjunto:
                        #print(muertes)
                        #print(posible)
                        # ojo con el orden de esto
                        if not len(list(filter(lambda muerte: muerte.issubset(posible), muertes))):
                            nuevo_conjunto.append(posible)
                            #print('No estaba muerto \n')

                        else:
                            pass
                            #print('si estaba muerto')
                            #print(nuevo_conjunto)
                            #print('---------\n')
            except IndexError: 
                #print('\n index error')
                #print('\nMe demoré {} seg en el join'.format((datetime.now() - tiempo2).total_seconds()))

                return nuevo_conjunto
    #print(nuevo_conjunto)
    #print('\nMe demoré {} seg en el join'.format((datetime.now() - tiempo2).total_seconds()))
    return nuevo_conjunto

def join_conjunto_sets2(conjuntos, muertes):
    aux = []
    nuevo_conjunto = []
    siguiente_conjunto = 1
    tiempo = datetime.now()
    
    #t = datetime.now()
    for A in conjuntos[:-1]:
        for B in conjuntos[siguiente_conjunto:]:
            if A - {max(A)} == B - {max(B)}:
                posible = A.union(B)
                muere = False
                aux.append(posible)
                for muerte in muertes:
                    if muerte.issubset(posible):
                        muere = True
                        break
                if not muere:
                    nuevo_conjunto.append(posible)
            else:
                continue

                #if not len(list(filter(lambda muerte: all(dato in C for dato in muerte), muertes))):
                 #   nuevo_conjunto.append(C) 
        siguiente_conjunto += 1
        
    #print('2.- for de sobre umbral tardo {}\n'.format((datetime.now() - t).total_seconds()))

    #print('\nMe demoré {} seg en el join'.format((datetime.now() - t).total_seconds()))
    #print('Join entre \n{}\nCon muertes: {}\ngenero:\n{} \n\n'.format(conjuntos,muertes, nuevo_conjunto))         
    
    return nuevo_conjunto

In [4]:
def fit(frec_min, datos='spotify.npy'):
    tiempo = datetime.now()
    data = np.load(datos)
    listas_datos = [set(x) for x in dict(data.item()).values()]
    frec_min = frec_min * len(listas_datos)
    
    canciones = [x for songs in listas_datos for x in songs]
    orden = list(set(canciones))

    enumeracion_canciones = dict(enumerate(orden))  # {0: cancion1, 1: cancion2}...
    canciones_enumeracion = {x[1]: x[0] for x in enumeracion_canciones.items()}  # {cancion1: 0, cancion2: 1... }
    
    tuplas = list(map(lambda dato: tuple(sorted(list(canciones_enumeracion[x] for x in dato))), listas_datos))
    
    
    #tuplas = (map(lambda dato: sorted(list(canciones_enumeracion[x] for x in dato)), listas_datos))
    # tuplas esta ordenada en orden ascendente de menor a mayor

    conjuntos = [{x} for x in canciones_enumeracion.values()]
    #conjuntos = ([x] for x in canciones_enumeracion.values())

    '''####################
    
    data = {'A':[1, 2, 5], 'B':[2, 4], 'c':[2, 3], 'd':[1, 2, 4], 'e':[1, 3], 'f':[2, 3], 
                                              'g':[1, 3], 'h':[1, 2, 3, 5], 'i':[1, 2, 3]}
    
    listas_datos = [x for x in data.values()]
    frec_min = frec_min * len(listas_datos)
    
    orden = list(set([x for songs in listas_datos for x in songs]))
    
    enumeracion_canciones = dict(enumerate(orden))  # {0: cancion1, 1: cancion2}...
    canciones_enumeracion = {x[1]: x[0] for x in enumeracion_canciones.items()}  # {cancion1: 0, cancion2: 1... }
    tuplas = list(map(lambda dato: tuple(sorted(list(canciones_enumeracion[x] for x in dato))), listas_datos))

    conjuntos = [{x} for x in canciones_enumeracion.values()]
    
    ####################'''

    eliminados = []
    retorno = []
    '''print((datetime.now() - tiempo).total_seconds())
    print()
    print(' -- iteracion 1 --')'''
    tiempo = datetime.now()
    # primer conjunto    
    diccionario = defaultdict(int)
    diccionario = defaultdict(int)
    
    '''total = chain(*listas_datos)
    for x in total:
        diccionario[x] += 1'''
    
    total = chain(*tuplas)
    for x in total:
        diccionario[x] += 1
    
    
    #print('tuplas: {}\n\n'.format(tuplas))
    conjuntos = [{x[0]} for x in (filter(lambda x: x[1] >= frec_min, diccionario.items()))]

    #tuplas2 = [x for x in (filter(lambda k: any(conjuntos) in k, tuplas))]
    
    tuplas2 = []
    nueva_tupla = []
    for tupla in tuplas:
        aux = []
        listo = False
        for conjunto in conjuntos:
            if conjunto.issubset(tupla):
                aux.append(*conjunto)
                if not listo:### 
                    listo = True
                    tuplas2.append(tupla)
                #break
        if aux: 
            nueva_tupla.append(tuple(aux))
    
    tuplas = tuplas2
    
    

    '''nueva_tupla = []
    frecuentes = list(chain(*tuplas2))
    for tupla in tuplas2:
        aux = []
        for elemento in tupla:
            if elemento in frecuentes:
                aux.append(elemento)
        nueva_tupla.append(tuple(aux))'''
    
    '''nueva_tupla = []
    for tupla in tuplas2:
        aux = []
        for conjunto in conjuntos:
            if conjunto.issubset(tupla):
                aux.append(conjunto)
        nueva_tupla.append(tuple(aux))'''
        
    '''print(nueva_tupla[:1])
    print()
    print(conjuntos)
    print()
    print(tuplas2[:2])'''

                
    
    
    '''print('Conjuntos:', type(conjuntos), conjuntos)
    print('tuplas:', type(tuplas), tuplas)
    print()'''
    k = 2
    #print((datetime.now() - tiempo).total_seconds())
    tiempo = datetime.now()
    while True:
        #print(' -- iteracion {} --'.format(k))
        k += 1
        t = datetime.now()
        tuplas, listas, muerte = conjuntos_sobre_umbral_sets(tuplas, conjuntos, frec_min)
        #print('tiempo sobre umbral:', (datetime.now() - t).total_seconds())

        t = datetime.now()
        conjuntos = join_conjunto_sets2(listas, muerte)
        #print('tiempo join:', (datetime.now() - t).total_seconds())

        #print('conjuntos:', conjuntos)

        if not listas:  
            return retorno
       

        else:
            aux = []
            for conjunto in listas:
                l = []
                for dato in conjunto:
                    l.append(enumeracion_canciones[dato])
                aux.append(l)

            retorno.extend(aux)
        #print((datetime.now() - tiempo).total_seconds())
        tiempo = datetime.now()

        #print()

In [5]:
t = 0
tiempo = datetime.now()
a = fit(0.01)

t += (datetime.now() - tiempo).total_seconds()
print('tiempo promedio:', t/10)

tiempo promedio: 459.71081569999996


In [6]:
print(a)

[['Where Is The Love?'], ['Yeah!'], ['Party In The U.S.A.'], ['Say My Name'], ['All The Small Things'], ['Paper Planes'], ["Stacy's Mom"], ['Forever'], ['Toxic'], ['Miss Independent'], ['Mr. Brightside'], ['Whatcha Say'], ['Replay'], ["It Wasn't Me"], ['Promiscuous'], ['Closer'], ['See You Again'], ['Dirty Little Secret'], ['Crazy In Love'], ['Hey Ya! - Radio Mix / Club Mix'], ['Home'], ['Man Of The Year'], ['Stronger'], ['Lean On (feat. MØ & DJ Snake)'], ['You & Me - Flume Remix'], ['Billie Jean'], ['No Role Modelz'], ['Snow (Hey Oh) - Hey Oh Album Version'], ['Do It Again'], ['Seven Nation Army'], ['Changes'], ['Juicy'], ['Hypnotize - 2014 Remastered Version'], ['Doses & Mimosas'], ['More Than a Feeling'], ["Jessie's Girl"], ['Work REMIX'], ['Bohemian Rhapsody - Remastered 2011'], ['Ignition - Remix'], ['The Next Episode'], ['ABC'], ['Free Bird'], ['No Type'], ["Don't Stop Believin'"], ['1985'], ['Sympathy For The Devil'], ['Sweet Emotion'], ['Big Poppa'], ['Summertime'], ['Circles']

In [733]:
a1 = [['Forever'], ['Say My Name'], ['Closer'], ['Home'], ['Lean On (feat. MØ & DJ Snake)'], ['Stronger'], ["Don't Stop Believin'"], ['Ignition - Remix'], ['No Role Modelz'], ['Riptide'], ['Mercy'], ['One Dance'], ['Redbone'], ['My Girl'], ['Body Like A Back Road'], ["I'm the One"], ['Despacito - Remix'], ['Hello'], ['Bounce Back'], ['goosebumps'], ['Unforgettable'], ['XO TOUR Llif3'], ['White Iverson'], ['Black Beatles'], ['Jumpman'], ['Ni**as In Paris'], ['Congratulations'], ['Paris'], ['Let It Go'], ['Hallelujah'], ['Blessings'], ['Skinny Love'], ['Fake Love'], ['Caroline'], ["That's What I Like"], ['Shape of You'], ['Say It'], ['Starboy'], ['Slide'], ['Alright'], ['T-Shirt'], ['September'], ['Gold'], ['Down'], ['Let Me Love You'], ['Stay'], ['Location'], ['1-800-273-8255'], ["Don't"], ['Sorry'], ['Hurricane'], ['Roses'], ['Ride'], ['Trap Queen'], ['HUMBLE.'], ['Passionfruit'], ["Don't Let Me Down"], ['Panda'], ['Broccoli (feat. Lil Yachty)'], ['Bad and Boujee (feat. Lil Uzi Vert)'], ['Mask Off'], ['No Problem (feat. Lil Wayne & 2 Chainz)'], ['Antidote'], ['Work'], ['iSpy (feat. Lil Yachty)'], ['Hotline Bling'], ['The Hills'], ['Gold Digger']]
a2 = [['Forever'], ['Say My Name'], ['Closer'], ['Home'], ['Lean On (feat. MØ & DJ Snake)'], ['Stronger'], ["Don't Stop Believin'"], ['Ignition - Remix'], ['No Role Modelz'], ['Riptide'], ['Mercy'], ['One Dance'], ['Redbone'], ['My Girl'], ['Body Like A Back Road'], ["I'm the One"], ['Despacito - Remix'], ['Hello'], ['Bounce Back'], ['goosebumps'], ['Unforgettable'], ['XO TOUR Llif3'], ['White Iverson'], ['Black Beatles'], ['Jumpman'], ['Ni**as In Paris'], ['Congratulations'], ['Paris'], ['Let It Go'], ['Hallelujah'], ['Blessings'], ['Skinny Love'], ['Fake Love'], ['Caroline'], ["That's What I Like"], ['Shape of You'], ['Say It'], ['Starboy'], ['Slide'], ['Alright'], ['T-Shirt'], ['September'], ['Gold'], ['Down'], ['Let Me Love You'], ['Stay'], ['Location'], ['1-800-273-8255'], ["Don't"], ['Sorry'], ['Hurricane'], ['Roses'], ['Ride'], ['Trap Queen'], ['HUMBLE.'], ['Passionfruit'], ["Don't Let Me Down"], ['Panda'], ['Broccoli (feat. Lil Yachty)'], ['Bad and Boujee (feat. Lil Uzi Vert)'], ['Mask Off'], ['No Problem (feat. Lil Wayne & 2 Chainz)'], ['Antidote'], ['Work'], ['iSpy (feat. Lil Yachty)'], ['Hotline Bling'], ['The Hills'], ['Gold Digger']]

In [734]:
a1 == a2

True